<a href="https://colab.research.google.com/github/sberdine77/LSTM-aws-spot-price/blob/main/notebook/LSTM_aws_spot_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Necessary instalations
import sys
## Uncomment for MacOS
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install tensorflow-macos==2.9
#!{sys.executable} -m pip install tensorflow-metal==0.5.0
#!{sys.executable} -m pip install scikit-learn

!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install tensorflow-macos
!{sys.executable} -m pip install scikit-learn

In [2]:
#!{sys.executable} -m pip install numpy --upgrade
#Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [8]:
tf.random.set_seed(7)
df = pd.read_json('../data/spot-price-history-Linux-Unix.json')
df = df[['SpotPrice', 'Timestamp']]
df['Timestamp'] = df['Timestamp'].astype(str)
df['SpotPrice'] = df['SpotPrice'].astype("float32")
print(df)
#df['Timestamp'] = df['Timestamp'].replace('\+','', regex=True).astype(str)
#df['SpotPrice'] = df['SpotPrice'].astype(float)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
df['SpotPrice'] = scaler.fit_transform(df[['SpotPrice']])

dataset = df.values
#print(dataset)
#dataset = dataset.astype('float32')
#dataset = dataset.astype('float64')
print(dataset)

     SpotPrice                  Timestamp
0       0.0203  2023-02-27 21:05:20+00:00
1       0.0204  2023-02-27 18:18:36+00:00
2       0.0190  2023-02-27 18:18:36+00:00
3       0.0204  2023-02-27 15:54:57+00:00
4       0.0190  2023-02-27 06:29:19+00:00
..         ...                        ...
339     0.0194  2022-12-25 08:22:21+00:00
340     0.0190  2022-12-24 17:43:00+00:00
341     0.0190  2022-12-24 17:43:00+00:00
342     0.0190  2022-12-23 17:42:08+00:00
343     0.0190  2022-12-23 17:42:08+00:00

[344 rows x 2 columns]
[[0.0214521586894989 '2023-02-27 21:05:20+00:00']
 [0.02310234308242798 '2023-02-27 18:18:36+00:00']
 [0.0 '2023-02-27 18:18:36+00:00']
 [0.02310234308242798 '2023-02-27 15:54:57+00:00']
 [0.0 '2023-02-27 06:29:19+00:00']
 [0.024752497673034668 '2023-02-27 03:12:10+00:00']
 [0.02310234308242798 '2023-02-26 20:37:49+00:00']
 [0.0214521586894989 '2023-02-26 15:19:19+00:00']
 [0.018151819705963135 '2023-02-26 08:03:10+00:00']
 [0.0 '2023-02-26 06:29:08+00:00']
 [0.016501

In [9]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

230 114


In [10]:
def create_dataset(dataset, look_back=1):
  dataX, dataY = [], []
  for i in range(len(dataset)-look_back-1):
    a = dataset[i:(i+look_back), 0]
    dataX.append(a)
    dataY.append(dataset[i + look_back, 0])
  return np.asarray(dataX).astype('float32'), np.asarray(dataY).astype('float32')

look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [11]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Epoch 1/100


2023-03-22 21:10:37.420565: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-22 21:10:37.577355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-22 21:10:37.664192: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


228/228 - 3s - loss: 0.0610 - 3s/epoch - 11ms/step
Epoch 2/100
228/228 - 1s - loss: 0.0559 - 1s/epoch - 5ms/step
Epoch 3/100
228/228 - 1s - loss: 0.0559 - 1s/epoch - 5ms/step
Epoch 4/100
228/228 - 1s - loss: 0.0559 - 1s/epoch - 5ms/step
Epoch 5/100
228/228 - 1s - loss: 0.0561 - 1s/epoch - 5ms/step
Epoch 6/100
228/228 - 1s - loss: 0.0562 - 1s/epoch - 5ms/step
Epoch 7/100
228/228 - 1s - loss: 0.0559 - 1s/epoch - 5ms/step
Epoch 8/100
228/228 - 1s - loss: 0.0561 - 1s/epoch - 6ms/step
Epoch 9/100
228/228 - 1s - loss: 0.0560 - 1s/epoch - 6ms/step
Epoch 10/100
228/228 - 1s - loss: 0.0558 - 1s/epoch - 5ms/step
Epoch 11/100
228/228 - 1s - loss: 0.0560 - 1s/epoch - 5ms/step
Epoch 12/100
228/228 - 1s - loss: 0.0558 - 1s/epoch - 5ms/step
Epoch 13/100
228/228 - 1s - loss: 0.0560 - 1s/epoch - 5ms/step
Epoch 14/100
228/228 - 1s - loss: 0.0559 - 1s/epoch - 5ms/step
Epoch 15/100
228/228 - 1s - loss: 0.0560 - 1s/epoch - 5ms/step
Epoch 16/100
228/228 - 1s - loss: 0.0560 - 1s/epoch - 5ms/step
Epoch 17/100

2023-03-22 21:12:50.381833: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-22 21:12:50.496216: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 0s 8ms/step
Train Score: 0.01 RMSE
Test Score: 0.02 RMSE


In [ ]:
## Solve the time gaps problems before running this
# # shift train predictions for plotting
# trainPredictPlot = np.empty_like(dataset)
# trainPredictPlot[:, :] = np.nan
# trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# # shift test predictions for plotting
# testPredictPlot = np.empty_like(dataset)
# testPredictPlot[:, :] = np.nan
# testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# # plot baseline and predictions
# df['SpotPrice'] = scaler.inverse_transform(df[['SpotPrice']])
# dataset = df.values
# plt.plot(dataset)
# plt.plot(trainPredictPlot)
# plt.plot(testPredictPlot)
# plt.show()